In [ ]:
#Load libraries 
import pandas as pd
import sspa
import seaborn as sns
import matplotlib.pyplot as plt
import scipy


In [ ]:
df = pd.read_csv('../Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_gmt("../Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

In [ ]:
df = pd.read_csv('../Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = '../Data/UniProt2Reactome_all_Levels_ver84.txt', download_latest = False, filepath = None)

In [ ]:
#Convert pathway ID to name
root_path = pd.read_excel('../Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}

root_pathway_names = list(root_pathway_dict.keys())
#Remove root pathways
reactome_pathways = reactome_pathways[~reactome_pathways.index.isin(root_pathway_names)]

In [ ]:
for i in range(len(df.index)):
    if df.WHO_status[i] == '1-2':
        df['Group'][i] = 'Mild'
    else:
        df['Group'][i] = 'Severe'


### Over-representation analysis

In [ ]:
#Initiate an ORA object 
ora = sspa.sspa_ora(df.iloc[:,:-2], df["Group"], reactome_pathways, 0.05, custom_background=None)
#Carry out ORA
ora_res = ora.over_representation_analysis()
print("Number of differentially abundant molecules", len(ora.DA_molecules))

display(ora.DA_test_res.sort_values(by="P-value"))
display(ora_res.sort_values(by="P-value"))


top_10_pathways = ora_res.sort_values(by="P-value").iloc[0:10, :]

In [ ]:
#Change name to have a line break otherwise it won't fit on the plot
top_10_pathways.Pathway_name[62] = '\n Transport of inorganic cations/anions and \n amino acids/oligopeptides   '

In [ ]:
plt.figure(figsize=(9, 5))

sns.set(font_scale = 2)
sns.set_style("ticks") # same as "white" but with ticks

#If you want to colour by significance
#bar_color = ['tab:green' if float(i) < 0.05 else 'tab:grey' for i in top_20_pathways['P-value']]
#sns.barplot(data=top_10_pathways, y="Pathway_name", x="P-value", orient="h", palette=bar_color) #rocket, magma

#If you want to colour by gradient
ax = sns.barplot(data=top_10_pathways, y="Pathway_name", x="P-value", orient="h", palette="rocket") #rocket, magma

#Increase space between the bars and the axis
ax.relim()
ax.autoscale_view()


#Add pathway coverage
label = []
for i in top_10_pathways.Coverage:
    num = i.split('/')
    print(num)
    percent = (int(num[0]) / int(num[1])) *100
    label.append("  "+ str(round(percent,1))+ '%')

print(label)

ax.bar_label(ax.containers[0], labels=label)

#plt.title('Metabolomics',fontsize=22,pad=10)
plt.xlabel('Unadjusted p-value',fontsize=26)
plt.ylabel('Pathway name',fontsize=26) 
plt.xlim(0, 0.2)


plt.axvline(0.05, c="black")


#plt.savefig( '../Figures/proteomic_ORA_top_10.png' , dpi=300,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
list(top_10_pathways.Coverage)

For integrated data, take the two ORA results and combine the p-values with Fisher's method, which is what is commonly done (See Maghsoudi et al., 2021 for examples):

In [ ]:
metabolomic_ora = ora_res.sort_values(by="P-value")

In [ ]:
proteomic_ora = ora_res.sort_values(by="P-value")

In [ ]:
metabolomic_ora.index = metabolomic_ora["ID"]
metabolomic_ora.drop(columns = ["ID"]) #using Sara's code to drop root pathways

In [ ]:
proteomic_ora.index = proteomic_ora["ID"]
proteomic_ora.drop(columns = ["ID"]) #using Sara's code to drop root pathways

In [ ]:
result = metabolomic_ora.merge(proteomic_ora, how='inner',right_index=True, left_index = True)
#"inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.""conventional_PA copy.ipynb"
result[:5]

In [ ]:
#Combine p-values

result["combined_pval"] = 0

for i in range(len(result.index)):
    ID_list = [result["P-value_x"][i],result["P-value_y"][i]]
    print(ID_list)
    test_stat,pval = scipy.stats.combine_pvalues(ID_list, method='fisher',weights=None)
    print(pval)
    result["combined_pval"][i] = pval


In [ ]:
result[:10]

In [ ]:
top_10_pathways = result.sort_values(by="combined_pval").iloc[0:10, :]

For the single omics, percentage coverage can be calculated easily from the 'Coverage' column. For the integrated dataset, you need to find the Reactome pathway definitions.

In [ ]:
#Obtain pathway coverage for the top 10 results
df = pd.read_csv("../Data/Su_integrated_data.csv", index_col=0)
reactome_pathways = pd.read_csv("../Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str") #Dtype warning because in some columns, some values are in string format whereas some are in integer format, that's why I specify dtype="str"
kpca_scores = sspa.sspa_kpca(df.iloc[:,:-2], reactome_pathways)


#Convert pathway ID to name
root_path = pd.read_excel('../Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}

root_pathway_names = list(root_pathway_dict.keys())
#Using Sara's code, remove root pathways
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))



#Filter out the molecules in the pathways that are not present in the dataset
#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#Obtain pathways and corresponding molecules for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}


#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:

plt.figure(figsize=(9, 5.5))
sns.set(font_scale = 2)

sns.set_style("ticks") # same as "white" but with ticks

ax = sns.barplot(data=top_10_pathways, y="Pathway_name_x", x="combined_pval", orient="h", palette="rocket") #rocket, magma


#Increase space between the bars and the axis
ax.relim()
ax.autoscale_view()



label = []
# percent1 = []
# percent2 = []

#Getting pathway coverage by taking the average of the metabolomic and proteomic pathway coverage (not ideal)
# for i in range(10):
#     pathway1 =  top_10_pathways.Coverage_x[i]
#     num = pathway1.split('/')
#     percent1 = (int(num[0]) / int(num[1])) *100
#     print(percent1)

#     pathway2 =  top_10_pathways.Coverage_y[i]
#     num = pathway2.split('/')
#     percent2 = (int(num[0]) / int(num[1])) *100
#     print(percent2)

#     percent = (percent1+percent2)/2
#     print(percent)
#     label.append("  "+ str(round(percent,1))+ '%')

# print(label)


#Getting pathway coverage by using the Reactome pathway definitions concatenated together
for i in range(10):
    pathway_nam = top_10_pathways.iloc[i,0]
    
    num_in_df = len(filtered_dict[pathway_nam])
    num_whole_pathway = len(orig_dict[pathway_nam])

    percent = (num_in_df/num_whole_pathway) * 100

    label.append("  "+ str(round(percent,1))+ '%')

ax.bar_label(ax.containers[0], labels=label)

#plt.title('ORA for integrated data',fontsize=22,pad=10)
plt.xlabel('Unadjusted p-value',fontsize=26)
plt.ylabel('Pathway name',fontsize=26) 
plt.xlim(0, 0.57);

plt.axvline(0.05, c="black")

#plt.savefig( '../Figures/integrated_ORA_top_10.png' , dpi=300,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

### Gene-set Enrichment Analysis

In [ ]:
gsea_res = sspa.sspa_gsea(df.iloc[:,:-2], df["Group"], reactome_pathways)

In [ ]:
display(gsea_res.sort_values(by="P-adjust FDR")) #0.54 is the lowest for metabolomic, 0.056 for proteomic
#display(gsea_res.sort_values(by="P-value"))

In [ ]:
#Change name to have a line break otherwise it won't fit on the plot
gsea_res.Pathway_name[0] = 'Regulation of Insulin-like Growth Factor (IGF) transport and \n uptake by Insulin-like Growth Factor Binding Proteins (IGFBPs)'

In [ ]:
top_10_pathways_gsea = gsea_res.sort_values(by="P-value").iloc[0:10, :]
top_10_pathways_gsea.index = range(len(top_10_pathways_gsea.index))

For integrated data, take the two GSEA results and combine the p-values with Fisher's method, which is what is commonly done (See Maghsoudi et al., 2021 for examples):

In [ ]:
metabolomic_gsea = gsea_res.sort_values(by="P-value")
metabolomic_gsea = metabolomic_gsea.set_index(['Pathway_ID'])

In [ ]:
proteomic_gsea = gsea_res.sort_values(by="P-value")
proteomic_gsea = proteomic_gsea.set_index(['Pathway_ID'])

In [ ]:
metabolomic_gsea

In [ ]:
proteomic_gsea

In [ ]:
result = metabolomic_gsea.merge(proteomic_gsea, how='inner',right_index=True, left_index = True)
result[:5]

In [ ]:
result["combined_pval"] = 0

In [ ]:
for i in range(len(result.index)):
    ID_list = [result["P-value_x"][i],result["P-value_y"][i]]
    print(ID_list)
    test_stat,pval = scipy.stats.combine_pvalues(ID_list, method='fisher',weights=None)
    print(pval)
    result["combined_pval"][i] = pval


In [ ]:
top_10_pathways_gsea = result.sort_values(by="combined_pval").iloc[0:10, :]

For ORA, percentage coverage can be calculated easily from the 'Coverage' column. For GSEA, you need to find the Reactome pathway definitions. (ENTITY% IS NOT PATHWAY COVERAGE)

In [ ]:
#Obtain pathway coverage for the top 10 results
df = pd.read_csv('../Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_gmt("../Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")  #2294 Reactome pathways

#df = pd.read_csv('../Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
#reactome_pathways = sspa.process_reactome('Homo sapiens', infile = '../Data/UniProt2Reactome_All_Levels_ver84.txt', download_latest = False, filepath = None) #2596 Reactome pathways

#df = pd.read_csv("../Data/Su_integrated_data.csv", index_col=0)
#reactome_pathways = pd.read_csv("../Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str") #Dtype warning because in some columns, some values are in string format whereas some are in integer format, that's why I specify dtype="str"



kpca_scores = sspa.sspa_kpca(df.iloc[:,:-2], reactome_pathways)


In [ ]:
#Convert pathway ID to name
root_path = pd.read_excel('../Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}

root_pathway_names = list(root_pathway_dict.keys())
#Using Sara's code, remove root pathways
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))


In [ ]:

#Filter out the molecules in the pathways that are not present in the dataset
#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#Obtain pathways and corresponding molecules for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}


#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

Code for metabolomic and proteomic plot:

In [ ]:
from matplotlib.lines import Line2D #To colour the bar by enrichment for the legend

plt.figure(figsize=(10, 7))

sns.set(font_scale = 2) #sns.set(font_scale = 1.2) for metabolomic because of long label
sns.set_style("ticks") # same as "white" but with ticks

#Set bar colour based on normalised enrichment score sign
bar_color = ['tab:red' if float(i) > 0 else 'tab:blue' for i in top_10_pathways_gsea['NES']]
ax = sns.barplot(data=top_10_pathways_gsea, y="Pathway_name", x="P-value", orient="h", palette=bar_color)

#Increase space between the bars and the axis
ax.relim()
ax.autoscale_view()


label=[]

for i in range(10):

    pathway_nam = top_10_pathways_gsea.Pathway_ID[i]
    
    num_in_df = len(filtered_dict[pathway_nam])
    num_whole_pathway = len(orig_dict[pathway_nam])
    
    print(pathway_nam)
    print(filtered_dict[pathway_nam])
    print(orig_dict[pathway_nam])

    percent = (num_in_df/num_whole_pathway) * 100

    label.append("  "+ str(round(percent,1))+ '%')


print(label)



ax.bar_label(ax.containers[0], labels=label)

#plt.title('GSEA for integrated data',fontsize=22, pad=10)
plt.xlabel('Unadjusted p-value',fontsize=26)
plt.ylabel('Pathway name',fontsize=26) 
plt.xlim(0, 0.21);

plt.axvline(0.05, c="black")

#Add legend
custom_lines = [Line2D([0], [0], color='tab:red', lw=4),
                Line2D([0], [0], color='tab:blue', lw=4)]
plt.legend(handles=custom_lines, labels=['Positive enrichment score', 'Negative enrichment score'],loc="upper right")


#plt.savefig( '../Figures/integrated_GSEA_top_10.png' , dpi=300,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

Code for integrated plot:

In [ ]:
from matplotlib.lines import Line2D #To colour the bar by enrichment for the legend

plt.figure(figsize=(10, 7))

sns.set(font_scale = 2) #sns.set(font_scale = 1.2) for metabolomic because of long label
sns.set_style("ticks") # same as "white" but with ticks

#Set bar colour based on normalised enrichment score sign
bar_color = ['tab:red' if float(i) > 0 else 'tab:blue' for i in top_10_pathways_gsea['NES_x']]
ax = sns.barplot(data=top_10_pathways_gsea, y="Pathway_name_x", x="combined_pval", orient="h", palette=bar_color)

#Increase space between the bars and the axis
ax.relim()
ax.autoscale_view()


label=[]
# for i in range(10):
#     pathway1 =  top_10_pathways_gsea["Entity %_x"][i][:-1]
    
#     pathway2 =  top_10_pathways_gsea["Entity %_y"][i][:-1]

#     percent = (float(pathway1)+float(pathway2))/2
#     print(percent)
#     label.append("  "+ str(round(percent,1))+ '%')

# print(label)

for i in range(10):

    pathway_nam = top_10_pathways_gsea.index[i]
    
    num_in_df = len(filtered_dict[pathway_nam])
    num_whole_pathway = len(orig_dict[pathway_nam])

    percent = (num_in_df/num_whole_pathway) * 100

    label.append("  "+ str(round(percent,1))+ '%')


print(label)



ax.bar_label(ax.containers[0], labels=label)

#plt.title('GSEA for integrated data',fontsize=22, pad=10)
plt.xlabel('Unadjusted p-value',fontsize=26)
plt.ylabel('Pathway name',fontsize=26) 
plt.xlim(0, 0.21);

plt.axvline(0.05, c="black")

#Add legend
custom_lines = [Line2D([0], [0], color='tab:red', lw=4),
                Line2D([0], [0], color='tab:blue', lw=4)]
plt.legend(handles=custom_lines, labels=['Positive enrichment score', 'Negative enrichment score'],loc="upper right")


#plt.savefig( '../Figures/integrated_GSEA_top_10.png' , dpi=300,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')